<a href="https://colab.research.google.com/github/Berenice2018/DeepLearning/blob/master/PySyft_Simple_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports, setup


In [0]:
!pip install syft

In [2]:
import time
import datetime
import logging
import math

import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset

import syft as sy

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

logger = logging.getLogger(__name__)

print(torch.cuda.is_available())

W0722 19:15:48.256587 140064218503040 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0722 19:15:48.277531 140064218503040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



True


In [3]:
# save the model on Google Drive, link Google drive to this notebook
from google.colab import drive
drive.mount('/content/gdrive')


# After executing this cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/gdrive/My Drive/Colab Notebooks/flower_data/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
picco_test  test  train  train_ofgdrive  valid	valid_ofgdrive


In [0]:
# paths to training and test data
data_dir = '/content/gdrive/My Drive/Colab Notebooks/flower_data/'
train_dir = data_dir + 'train'
valid_dir = data_dir + 'valid'

#os.chdir("/content/gdrive/My Drive/Colab Notebooks/")
test_dir = data_dir + 'test'

### Architecture and helpers

In [0]:
# Make data loader based on the selected pre-trained model
def create_loaders(base, final = False):
    print('returning datasets')
    # ResNet, DenseNet expect 224, Inception expects 299
    img_size = 299 if base == 'Inception' else 224 

    transforms_train = transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

    transforms_test = transforms.Compose([
        transforms.Resize(img_size + 1),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

    # Load the datasets with ImageFolder
    trainset = datasets.ImageFolder(train_dir, transform=transforms_train)
    validationset = datasets.ImageFolder(valid_dir, transform=transforms_train)
    testset = datasets.ImageFolder(valid_dir, transform=transforms_test)
       
    return trainset, validationset, testset

In [0]:
#transforms a torch.Dataset or a sy.BaseDataset into a sy.FederatedDataset. 
def dataset_federate(dataset, workers):
    print('dataset_federate')

    datasets = []
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=32)
    
    for dataset_idx, (data, targets) in enumerate(data_loader):
        worker = workers[dataset_idx % len(workers)]
        data = data.send(worker)
        targets = targets.send(worker)
        datasets.append(sy.BaseDataset(data, targets))
    
    fed_dataset = sy.FederatedDataset(datasets)
    fed_loader = sy.FederatedDataLoader(fed_dataset, batch_size=32, shuffle=False, drop_last=False)
    
    return fed_loader

In [0]:
class ClassifierH1(nn.Module):
    def __init__(self, inp = 784, h1=512, out = 10, d=0.3):
        super(ClassifierH1, self).__init__()
        self.fc1 = nn.Linear(inp, h1)
        self.fc2 = nn.Linear(h1, out)
        
        self.dropout = nn.Dropout(d)
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)        
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [0]:
# Helper functions to have flexibility with choice if needed
# The optimizer to be used for training
def create_optimizer(params, opt, lr):
    return optim.SGD(params, lr= lr)

# Scheduler that automatically adjusts learning rate of optimizer
def create_scheduler(opt, p=7, f = 0.1):
    return optim.lr_scheduler.ReduceLROnPlateau(opt, patience = p, factor=f)

# Loss function selector
def create_loss(ls):
    return nn.CrossEntropyLoss() if ls == 'Entr' else nn.NLLLoss()

In [0]:
class MyNetwork():
       
    def __init__(self, base, clf, kwargs):
        # Additional variables will be used to track training progress for easier re-loads, plotting, etc.
        self.last_epoch = 0       # Keeps track of number of epochs trained
        self.min_vloss = np.inf   # Keeps track of lowest validation loss
        self.lr_hist = []         # Tracks learn rate changes over epochs 
        self.vloss_hist = []      # Tracks validation loss changes over epochs
        self.tloss_hist = []      # Tracks training loss changes over epochs
        self.acc_hist = []        # Tracks accuracy rate changes over epochs
        self.opt_dict = None      # Stores optimizer dictionary
        self.sch_dict = None      # Stores scheduler dictionary
        self.sch_wait = None      # Stores scheduler "patience" parameter
        self.clf_name = clf       # Stores the name of the chosen classifier
        self.base_name = base     # Stores the name of chosen CNN
        self.loss_name = None     # Stores the name of chosen loss function
        self.opt_name = None      # Stores the name of chosen optimizer method
        self.layer_dict = kwargs  # Stores the info on number of nodes in each clf layer, as well as dropout %
        # When the datasets are created, each class value automatically got assigned an index.
        # These will be created and stored in the training function 
        self.class_to_idx = None
        self.idx_to_class = None
        
        # Create vanilla feature net and classifier
        base_net = self.import_model(base)
        my_clf = self.create_clf(base, clf, kwargs)

        # Adding the classifier to pre-trained network (replacing last layer)
        if base == 'ResNet152':
            base_net.fc = my_clf
        elif base == 'Inception':
            base_net.fc = my_clf
        elif base == 'DenseNet161':
            base_net.classifier = my_clf
            
        self.my_net = base_net 
        self.my_clf = my_clf
        
        print('Model created from base_net {}, Clf {} with {}'.format(base, clf, kwargs))
        
    # Pre-trained conv-net used for feature extraction before final classification
    def import_model(self, mod, tr = True):
        if mod == 'ResNet152':
            m = models.resnet152(pretrained=tr)
        elif mod == 'Inception':
            m = models.inception_v3(pretrained=tr) # aux_logits=False)
        elif mod == 'DenseNet161':
            m = models.densenet161(pretrained=tr)
        else:
            print('No base model imported!')
        
        # Keep the weights fixed on the pre-trained model?
        if tr:
            for param in m.parameters():
                param.requires_grad = True 
        
        return m     
        
    # Create a specific classifier
    def create_clf(self, base, clf, kwargs):
        # Define input size, based on the output of pre-trained models:
        if base == 'ResNet152':
            in_size = 2048
        elif base == 'Inception':
            in_size = 2048
        elif base == 'DenseNet161':
            in_size = 2208
        if clf == 'H1':
            return ClassifierH1(in_size, kwargs['h1'], kwargs['out'], kwargs['d'])
        else:
            print('Classifier not found')
 

In [0]:
def train_model(model, n_epochs, isFinal, train_params = None):
    criterion = create_loss(train_params['loss'])
    optimizer = create_optimizer(model.my_clf.parameters(), train_params['opt'], train_params['lr'])
    scheduler = create_scheduler(optimizer, train_params['wait'])
    model.loss_name = train_params['loss']
    model.opt_name = train_params['opt']
    model.sch_wait = train_params['wait']
    model.vloss_min = np.inf
  
  # check if CUDA is available
    train_on_gpu = torch.cuda.is_available()
    if not train_on_gpu:
        print('CUDA is not available.  Training on CPU ...')
        device = "cpu"
    else:
        print('CUDA is available!  Training on GPU ...')
        device = "cuda"
        
    model.my_net.to(device)
    
    print('*** Training starting ***')
   
    
    for epoch in range(2):
      
      # keep track of training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        valid_acc = 0.0
        
        model.my_net.train()
        
        for batch_idx, (data, target) in enumerate(train_loader):
            # PySyft: send the model to the right location
            print('model sent to data.location {}'.format(data.location))
            model.my_net.send(data.location)

            
            data, target = data.to(device), target.to(device)

            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            
            output = model.my_net(data)
            loss = criterion(output, target)
            
            loss.backward()
            optimizer.step()
            
            # PySyft: get the smarter model back
            model.my_net.get()
            
            # update training loss
            syft_loss = loss.get() # PySyft: get the loss back
            train_loss += syft_loss.item() * data.size(0)
        
        print('   Train loss: \t{:.6f}'.format(train_loss))

      

### Instantiation,  hyperparams, model training

In [0]:
# File name for saving model
filename_drive_best = './densenet161_test.pt'
filename_drive_fin = './densenet161_test.pt'

# Defining network model 
# Base net can be 'Inception', 'ResNet152' or 'DenseNet161'

base_net = 'DenseNet161'   
clf = 'H1' # H1 has one hidden layer, H2 has two
layers = {'out':102, 'd':0.2, 'h1':1024, 'h2':512} # Input size is pre-determined and set based on chosen pre-trained model. d is for droput rate.

# Training parameters
epochs = 2

# If set to False, 20% of train set is reserved for validation, 
# and test set reserved for testing the trained model
isFinal = True 
train_params = {
                'opt':   'SGD',   # Optimizer to be used for training
                'loss':  '',  # or 'Entr' for CrossEntropyLoss. Loss function to be used for training
                'lr':    1e-3,    # Learning rate for training
                'wait':  7        # "patience" parameter for scheduler function from Pytorch.
                }

In [12]:
# Create a blank model or load an existing one
model = MyNetwork(base_net, clf, layers)


def unfreeze(model):
    for name, child in model.named_children():
        if name in ['denseblock4', 'denseblock3', 'denseblock2', 'denseblock1']: #, 'denseblock1']:
            print(name + ' is unfrozen')
            for param in child.parameters():
                param.requires_grad = True
        else:
            print(name + ' is unfrozen')
            for param in child.parameters():
                param.requires_grad = True

unfreeze(model.my_net.features)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/checkpoints/densenet161-8d451a50.pth
100%|██████████| 115730790/115730790 [00:01<00:00, 99358261.23it/s] 


Model created from base_net DenseNet161, Clf H1 with {'out': 102, 'd': 0.2, 'h1': 1024, 'h2': 512}
conv0 is unfrozen
norm0 is unfrozen
relu0 is unfrozen
pool0 is unfrozen
denseblock1 is unfrozen
transition1 is unfrozen
denseblock2 is unfrozen
transition2 is unfrozen
denseblock3 is unfrozen
transition3 is unfrozen
denseblock4 is unfrozen
norm5 is unfrozen


In [0]:
# create workers, 
hook = sy.TorchHook(torch)

ada = sy.VirtualWorker(hook, 'ada')
bob = sy.VirtualWorker(hook, 'bob')
cyd = sy.VirtualWorker(hook, 'cyd')

In [0]:
# Create the data loaders, federated PySyft loaders are returned
my_trainset, my_validset, _ = create_loaders(model.base_name, isFinal)
train_loader = dataset_federate(my_trainset, (ada,bob,cyd))
valid_loader = dataset_federate(my_validset, (ada,bob,cyd))

returning datasets
dataset_federate


### start the training

In [0]:
print(f'objects of ada= {len(ada._objects)}, bob= {len(bob._objects)}, cyd= {len(cyd._objects)}')


##### START THE TRAINING #### 
train_model(model, epochs, isFinal, train_params)

### Clear the worker

In [0]:
ada.clear_objects()
bob.clear_objects()
cyd.clear_objects()